In [ ]:
import pandas as pd
import scanpy as sc

import time

import requests
from bs4 import BeautifulSoup
import pandas as pd
from typing import List, Dict
from datetime import datetime
import os
import hashlib
import json
from pathlib import Path
import re


class WebScraper:
    def __init__(self, base_url: str, cache_dir: str = ".cache"):
        self.base_url = base_url
        self.cache_dir = Path(cache_dir)
        self.cache_dir.mkdir(exist_ok=True)

        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        })

    def _get_cache_path(self, url: str) -> Path:
        """Generate a cache file path for a given URL."""
        url_hash = hashlib.md5(url.encode()).hexdigest()
        return self.cache_dir / f"{url_hash}.html"

    def fetch_page(self, url: str) -> BeautifulSoup:
        """
        Fetch and parse a webpage with caching.

        Args:
            url (str): The URL to fetch

        Returns:
            BeautifulSoup: Parsed HTML content
        """
        cache_path = self._get_cache_path(url)

        # Try to load from cache first
        if cache_path.exists():
            with open(cache_path, 'r', encoding='utf-8') as f:
                print(f"Loading from cache: {url}")
                return BeautifulSoup(f.read(), 'html.parser')

        # If not in cache, fetch and store
        try:
            print(f"Downloading: {url}")
            response = self.session.get(url)
            response.raise_for_status()

            # Save to cache
            with open(cache_path, 'w', encoding='utf-8') as f:
                f.write(response.text)

            return BeautifulSoup(response.text, 'html.parser')

        except requests.exceptions.RequestException as e:
            print(f"Error fetching {url}: {str(e)}")
            raise

    def extract_data(self, soup: BeautifulSoup) -> List[Dict]:
        """
        Extract desired data from the parsed HTML.
        Override this method based on your specific needs.

        Args:
            soup (BeautifulSoup): Parsed HTML content

        Returns:
            List[Dict]: Extracted data as a list of dictionaries
        """
        # Example extraction - modify according to your needs
        data = []
        # Add your extraction logic here
        return data

    def save_to_csv(self, data: List[Dict], filename: str):
        """
        Save extracted data to CSV file.

        Args:
            data (List[Dict]): Data to save
            filename (str): Output filename
        """
        df = pd.DataFrame(data)
        df.to_csv(filename, index=False)
        print(f"Data saved to {filename}")


def extract_data(soup: BeautifulSoup) -> List[Dict]:
    """
    Extract all cancer case data including metadata fields.

    Args:
        soup (BeautifulSoup): Parsed HTML content

    Returns:
        List[Dict]: List of cases with their details
    """

    def extract_field(soup: BeautifulSoup, field_name: str) -> str:
        """Helper function to extract field value from table row"""
        field_cell = soup.find('td', string=f"{field_name}:")
        if field_cell:
            value_cell = field_cell.find_next('td')
            return value_cell.text.strip() if value_cell else None
        return None
    case_data = {}
    # Extract publication data (keeping the existing publication extraction)
    pub_header = soup.find('td', string='Publications:')
    if pub_header:
        pub_cell = pub_header.find_next('td', class_='text-justify')
        if pub_cell:
            for div in pub_cell.find_all('div'):
                link = div.find('a')
                if link:
                    url = link.get('href', '')
                    authors_text = link.text.strip()

                    last_name_match = re.search(r'\s*(\w+) et al\.', authors_text)
                    last_name = last_name_match.group(1) if last_name_match else None

                    span = div.find('span')
                    details = span.text.strip() if span else ''

                    year_match = re.search(r',\s*((?:19|20)\d{2})', details)
                    year = year_match.group(1) if year_match else None
                    case_data.update({
                                    'authors': authors_text,
                                    'author_last_name': last_name,
                                    'url': url,
                                    'details': details,
                                    'year': year,
                                    'full_citation': authors_text + details})
                    # Extract all the additional fields
    case_data.update({
            # Cancer and tumor details
            'cancer_type': extract_field(soup, 'Cancer type'),
            'cancer_type_abbr': extract_field(soup, 'Cancer type abbreviation'),
            'primary_site': extract_field(soup, 'Primary site'),
            'sampling_site': extract_field(soup, 'Sampling site'),
            'tumor_grade': extract_field(soup, 'Tumor grade'),
            'tumor_status': extract_field(soup, 'Tumor status'),
            'strategy': extract_field(soup, 'Strategy'),
            'protocol': extract_field(soup, 'Protocol'),
            'instrument': extract_field(soup, 'Instrument'),
            'GEO accession': extract_field(soup, 'GEO accession'),
            # Patient details
            'age': extract_field(soup, 'Donor age'),
            'gender': extract_field(soup, 'Donor gender'),

            # Treatment and additional info
            'treatment': extract_field(soup, 'Treatment'),
            'other_metadata': extract_field(soup, 'Other metadata')
        })
    return case_data


IndentationError: expected an indented block (1617220460.py, line 173)

In [ ]:
metadata = pd.read_csv("CancerSCEM-Browse-Table.csv").iloc[:, :-3]
project_counts = metadata["Project ID"].value_counts().to_frame()
project_counts.columns = ["n_samples"]

metadata = metadata.merge(project_counts, how="left", left_on="Project ID", right_index=True)
metadata = metadata[~metadata["Project ID"].duplicated(keep="first")]
metadata = metadata[metadata["n_samples"] >= 2]

# Example usage


results = []
for sample_id in metadata["Sample ID"]:
    print(sample_id)
    subset = metadata[metadata["Sample ID"]==sample_id]
    target_url = f"https://ngdc.cncb.ac.cn/cancerscem/sample-details/{sample_id}"
    try:
        scraper = WebScraper(target_url, cache_dir=".cache")

        # Fetch and parse the webpage (will use cache if available)
        soup = scraper.fetch_page(target_url)

        # Extract data
        data = extract_data(soup)
        if data is not None:
            data["sample ID"] = sample_id
            data["n_samples"] = subset["n_samples"].item()
            data["Accession No."] = subset["Accession No."].item()
            data["Project ID"] = subset["Project ID"].item()
            results.append(data)
        else:
            print("Edge case", sample_id)
    except Exception as e:
        print(f"Scraping failed: {str(e)}")
results = pd.DataFrame(results)
results = results.drop(["age", "gender", "treatment", "sample ID"], axis=1)
results = results.set_index("Project ID")
results.to_csv("study_overview.csv")

In [2]:
_malignant_cell = "Malignant cells"
_cell_type = "cell_type"

In [3]:
study_overview = pd.read_csv("study_overview.csv", index_col=0)

In [4]:
study_overview = study_overview[(study_overview["Source"] == "CancerSCEM") & (study_overview["Include"] == "Yes")]

In [5]:
metadata = pd.read_csv("CancerSCEM-Browse-Table.csv").iloc[:, :-3]


In [6]:
metadata = metadata[metadata["Project ID"].isin(study_overview["Project ID"])]

In [7]:
study_overview

,Tissue,Article,Disease,Technology,N_samples,Source,author_last_name,year,Project ID,Include,Reason
Spalte 1,,,,,,,,,,,
Alchahin_2022_kidney,kidney,https://pubmed.ncbi.nlm.nih.gov/36180422\n3828...,RCC,10x,15,CancerSCEM,Alchahin,2022,ccRCC-061,Yes,NaN
Aubin_2022_brain,brain,https://pubmed.ncbi.nlm.nih.gov/35803925/,Pediatric ependymoma,Drop-seq,8,CancerSCEM,Aubin,2022,EPN-102,Yes,NaN
Barrett_2024_brain,brain,https://pubmed.ncbi.nlm.nih.gov/38216553/,Sporadic Vestibular Schwannomas,10x,15,CancerSCEM,Barrett,2024,VS-105,Yes,NaN
Becker_2022_colorectal,colorectal,https://pubmed.ncbi.nlm.nih.gov/35726067/,Colorectal cancer,10x,4,CancerSCEM,Becker,2022,CRC-067,Yes,NaN
Becker_2022_colorectal,colorectal,https://pubmed.ncbi.nlm.nih.gov/35726067/,Colorectal cancer,10x,5,CancerSCEM,Becker,2022,CRC-101,Yes,NaN
...,...,...,...,...,...,...,...,...,...,...,...
Zhang_2022_skin,skin,https://pubmed.ncbi.nlm.nih.gov/36433984/,Acral Melanoma,10x,7,CancerSCEM,Zhang,2022,AM-036,Yes,NaN
Zhang_2023_pancreas,pancreas,https://pubmed.ncbi.nlm.nih.gov/37612267/,PDAC,10x,7,CancerSCEM,Zhang,2023,PDAC-106,Yes,NaN
Zhou_2023_head-and-neck,head-and-neck,https://pubmed.ncbi.nlm.nih.gov/36879115/,Salivary Adenoid Cystic Carcinoma,10x,4,CancerSCEM,Zhou,2023,SACC-103,Yes,NaN


In [18]:

results = []
for sample_id in metadata["Sample ID"]:
    print(sample_id)
    subset = metadata[metadata["Sample ID"]==sample_id]
    target_url = f"https://ngdc.cncb.ac.cn/cancerscem/sample-details/{sample_id}"
    try:
        scraper = WebScraper(target_url, cache_dir=".cache")

        # Fetch and parse the webpage (will use cache if available)
        soup = scraper.fetch_page(target_url)

        # Extract data
        data = extract_data(soup)
        if data is not None:
            data["sample ID"] = sample_id
            data["Accession No."] = subset["Accession No."].item()
            data["Project ID"] = subset["Project ID"].item()
            results.append(data)
        else:
            print("Edge case", sample_id)
    except Exception as e:
        print(f"Scraping failed: {str(e)}")
results = pd.DataFrame(results)


LUAD-004-01-1A
Loading from cache: https://ngdc.cncb.ac.cn/cancerscem/sample-details/LUAD-004-01-1A
LUAD-004-02-1A
Loading from cache: https://ngdc.cncb.ac.cn/cancerscem/sample-details/LUAD-004-02-1A
LUAD-004-03-1A
Loading from cache: https://ngdc.cncb.ac.cn/cancerscem/sample-details/LUAD-004-03-1A
LUAD-004-06-1A
Loading from cache: https://ngdc.cncb.ac.cn/cancerscem/sample-details/LUAD-004-06-1A
LUAD-004-07-1A
Loading from cache: https://ngdc.cncb.ac.cn/cancerscem/sample-details/LUAD-004-07-1A
LUAD-004-08-1A
Loading from cache: https://ngdc.cncb.ac.cn/cancerscem/sample-details/LUAD-004-08-1A
LUSC-005-01-1A
Loading from cache: https://ngdc.cncb.ac.cn/cancerscem/sample-details/LUSC-005-01-1A
LUSC-005-02-1A
Loading from cache: https://ngdc.cncb.ac.cn/cancerscem/sample-details/LUSC-005-02-1A
LUSC-005-03-1A
Loading from cache: https://ngdc.cncb.ac.cn/cancerscem/sample-details/LUSC-005-03-1A
LUSC-005-04-1A
Loading from cache: https://ngdc.cncb.ac.cn/cancerscem/sample-details/LUSC-005-04-1A


In [21]:
results.to_csv("sample_metadata.csv")

In [8]:
results = pd.read_csv("sample_metadata.csv")
results = results[results["included"]=="yes"]

In [ ]:
from io import BytesIO
from typing import Optional

import requests
from scipy import sparse


def get_dataframe(url, compression = None) -> Optional[pd.DataFrame]:
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        content = BytesIO(response.content)
        df = pd.read_csv(content, compression=compression, sep='\t', index_col=0)
        return df


def get_anndata_from_sample_id(sample_id: str) -> Optional[sc.AnnData]:
    count_url = f'https://ngdc.cncb.ac.cn/cancerscem/downloads/expMatrixes/{sample_id}.counts.matrix.tsv.gz'
    celltype_url = f'https://ngdc.cncb.ac.cn/cancerscem/downloads/cell_annotation/{sample_id}.cell.type.txt'

    cell_type = get_dataframe(celltype_url)

    cell_type_counts = cell_type.value_counts()

    if not _malignant_cell in cell_type_counts.index:
        return 

    if cell_type_counts[_malignant_cell] < 50:
        return

    count_data = get_dataframe(count_url, compression="gzip")

    if (count_data is None) or (cell_type is None):
        return

    adata = sc.AnnData(count_data.transpose())
    adata.X = sparse.csr_matrix(adata.X)
    adata.obs = cell_type.loc[adata.obs_names].copy()
    adata.obs["sample_id"] = sample_id
    return adata



In [11]:
from genericpath import isfile
import pathlib

import tqdm

results_dir = pathlib.Path("data/CancerSCEM/")
skipped_samples = []
for _, (sample_id, project_id) in tqdm.tqdm(results[["sample ID", "Project ID"]].iterrows(), total=results.shape[0]):
    project_dir = results_dir.joinpath(project_id)
    if not project_dir.is_dir():
        project_dir.mkdir(parents=True)
    
    sample_path = project_dir.joinpath(f"{sample_id}.h5ad")
    if sample_path.is_file():
        continue
    adata = get_anndata_from_sample_id(sample_id)
    if adata is None:
        skipped_samples.append(sample_id)
        print(sample_id)
        continue
    adata.write_h5ad(sample_path)

 31%|███▏      | 232/737 [11:22<2:17:32, 16.34s/it]

WT-070-01-1A


 32%|███▏      | 233/737 [11:24<1:54:45, 13.66s/it]

WT-070-02-1A


 32%|███▏      | 234/737 [11:25<1:33:02, 11.10s/it]

...

 99%|█████████▉| 728/737 [4:50:03<04:47, 31.89s/it]

HSCC-130-10-1H


 99%|█████████▉| 731/737 [4:52:06<03:16, 32.71s/it]

CRC-133-01-1A


100%|██████████| 737/737 [4:54:15<00:00, 23.96s/it]


In [12]:
results.loc[results["sample ID"].isin(skipped_samples), ["included"]] = "no"
results.loc[results["sample ID"].isin(skipped_samples), ["reason"]] = "Too few malignant cells"
results = results.drop("Unnamed: 0", axis=1)
results.to_csv("sample_metadata.csv")

In [47]:
adata.var_names

Index(['AL627309.1', 'AL627309.5', 'AP006222.2', 'LINC01409', 'LINC01128',
       'LINC00115', 'FAM41C', 'AL645608.2', 'LINC02593', 'SAMD11',
       ...
       'MT-ND6', 'MT-CYB', 'BX004987.1', 'MAFIP', 'AC011043.1', 'AL354822.1',
       'AL592183.1', 'AC240274.1', 'AC233755.2', 'AC007325.4'],
      dtype='object', length=20506)

In [13]:
results["included"].value_counts()

included
yes    565
no     172
Name: count, dtype: int64

In [63]:
results.columns

Index(['Unnamed: 0', 'authors', 'author_last_name', 'url', 'details', 'year',
       'full_citation', 'cancer_type', 'cancer_type_abbr', 'primary_site',
       'sampling_site', 'tumor_grade', 'tumor_status', 'strategy', 'protocol',
       'instrument', 'GEO accession', 'age', 'gender', 'treatment',
       'other_metadata', 'sample ID', 'Accession No.', 'Project ID',
       'included', 'reason'],
      dtype='object')

In [3]:
import pathlib
import scanpy as sc

m_cells = 0
for project in pathlib.Path("data/CancerSCEM/").iterdir():
    print(project.stem)
    for dataset_path in project.iterdir():
        adata = sc.read_h5ad(dataset_path)
        m_cells += adata.obs["cell_type"].value_counts()[_malignant_cell]

TS-039
MPAL-022
TGCT-052
CM-036
LSCC-034
PDAC-044
UCEC-024
LUSC-092
BRCA-071
CRC-133
GBC-066
LUAD-096
GBM-077
PDAC-106
PDAC-132
RB-098
ESCC-079
OV-020
PAAD-038
ATC-026
PDAC-046
MB-078
UL-050
iCCA-066
CRC-086
SKCM-095
BLCA-115
PRAD-063
WT-070
CRC-030
ccRCC-061
ESCC-126
ccRCC-060
MPNST-114
MCC-122
GCTB-045
HCC-059
BRCA-135
ESCC-111
CaCx-120
SCE-112
HNSCC-087
HNSCC-127
SACC-103
HNSCC-121
MB-081
MPLC-068
HSCC-130
LUSC-005
EPN-102
RMC-125
VS-105
BRCA-107
EAC-116
BRCA-134
PDAC-064
GBM-117
PDAC-091
cSCC-099
BCC-109
NSCLC-006
MCC-084
AM-036
HNSCC-043
NSCLC-007
HGSOC-042
HGSOC-049
PDAC-051
PTC-069
MESO-065
GBM-085
LUAD-004
CRC-101
ccRCC-118
TNBC-075
PRAD-082
BRCA-128
BLCA-031
GCTB-052
CRC-067
ESCC-032
ESCC-048
GBM-080


In [4]:
m_cells

818940